# Runtime Test

For conver yolov5 model to trt plan (**yolov5s-2021-07-28.engine**) and build **libmyplugins.so** for your system see instruction: https://github.com/wang-xinyu/tensorrtx/tree/master/yolov5#how-to-run-yolov5s-as-example

Based on this instruction, we wrote a small shell script.

```bash
cd ../inference/convertors/yolo2tensorrt/bin/
./yolov5_tensorrt.sh
```
Before runing this script, please download install additionaly **opencv-contrib** and **pycuda**

   * Copy the resulting **libmyplugins.so** to the folder from which you will run this notebook
   * Replace the data model **data/models/Detector/yolov5engine/yolov5s-2021-07-28.engine** with the one you just got from running the script

Convert options and ocr
* options (inference/convertors/options2tensorrt/convert_numberplate_options_to_onnx.py)
* ocrs (inference/convertors/ocr2tensorrt/convert_ocr_to_onnx.py)

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
# Import all necessary libraries.
import sys
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time
import copy
from turbojpeg import TurboJPEG
import pycuda.autoinit

jpeg = TurboJPEG()

# NomeroffNet path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
from NomeroffNet.YoloV5EngineDetector import Detector

detector = Detector()
detector.load_model(
    engine_file_path="/var/www/nomeroff-net/inference/convertors/yolo2tensorrt/bin/yolov5s-2021-07-28.engine",
    plugin_lib="../inference/convertors/yolo2tensorrt/bin/libmyplugins.so")

In [4]:
from NomeroffNet.BBoxNpPoints import NpPointsCraft, getCvZoneRGB, convertCvZonesRGBtoBGR, reshapePoints

npPointsCraft = NpPointsCraft()
npPointsCraft.load()

Loading weights from checkpoint (/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_mlt/craft_mlt_25k_2020-02-16.pth)
Loading weights of refiner from checkpoint (/var/www/nomeroff-net/NomeroffNet/tools/../../data/./models/NpPointsCraft/craft_refiner/craft_refiner_CTW1500_2020-02-16.pth)


In [5]:
from NomeroffNet.OptionsDetectorOnnx import OptionsDetectorOnnx

optionsDetector = OptionsDetectorOnnx({
    "class_region": [
            "military",
            "eu_ua_2015",
            "eu_ua_2004",
            "eu_ua_1995",
            "eu",
            "xx_transit",
            "ru",
            "kz",
            "eu-ua-fake-dpr",
            "eu-ua-fake-lpr",
            "ge",
            "by",
            "su",
            "kg",
            "am"
        ],
        "count_lines": [
            1,
            2,
            3
        ],
})
optionsDetector.load_model("../data/model_repository/numberplate_options/1/model.onnx")

In [6]:
from NomeroffNet.TextDetectorOnnx import TextDetectorOnnx

# Initialize text detector.
textDetector = TextDetectorOnnx({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "../data/model_repository/ocr-eu_ua_2004_2015/1/model.onnx"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "../data/model_repository/ocr-eu_ua_1995/1/model.onnx"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "../data/model_repository/ocr-eu/1/model.onnx"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-ordlo-lpr", "eu-ua-ordlo-dpr"],
        "model_path": "../data/model_repository/ocr-ru/1/model.onnx"
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "../data/model_repository/ocr-kz/1/model.onnx"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "../data/model_repository/ocr-ge/1/model.onnx"
    },
    "su": {
        "for_regions": ["su"],
        "model_path": "../data/model_repository/ocr-su/1/model.onnx"
    }
}, default_label="eu")

In [7]:
async def test(dirName, fname):
    start_time = time.time()
    img_path = os.path.join(dirName, fname)
    with open(img_path, 'rb') as in_file:
        img = jpeg.decode(in_file.read())
    image_load_time = time.time() - start_time

    start_time = time.time()
    targetBoxes = detector.detect_bbox([img])
    detect_bbox_time = time.time() - start_time
    
    start_time = time.time()
    all_points = npPointsCraft.detect(img, targetBoxes)
    all_points = [ps for ps in all_points if len(ps)]
    craft_time = time.time() - start_time
    
    start_time = time.time()
    
    zones = convertCvZonesRGBtoBGR([getCvZoneRGB(img, reshapePoints(rect,1)) for rect in all_points])
    
    perspective_align_time = time.time() - start_time
    
    start_time = time.time()
    regionIds, countLines = optionsDetector.predict(zones)
    regionNames = optionsDetector.getRegionLabels(regionIds)
    classification_time = time.time() - start_time
    
    start_time = time.time()
    textArr = textDetector.predict(zones, regionNames, countLines)
    ocr_time = time.time() - start_time
    return (image_load_time, detect_bbox_time, craft_time, perspective_align_time, classification_time, ocr_time)

In [11]:
N = 10

i = 0
j = 0

image_load_time_all        = 0
detect_bbox_time_all       = 0
craft_time_all             = 0
perspective_align_time_all = 0
classification_time_all    = 0
ocr_time_all               = 0

start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            image_load_time, \
                detect_bbox_time, \
                craft_time, \
                perspective_align_time, \
                classification_time, \
                ocr_time = await test(dirName, fname)
            image_load_time_all        += image_load_time
            detect_bbox_time_all       += detect_bbox_time
            craft_time_all             += craft_time
            perspective_align_time_all += perspective_align_time
            classification_time_all    += classification_time
            ocr_time_all               += ocr_time
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
pass 1
pass 2
pass 3
pass 4
pass 5
pass 6
pass 7
pass 8
pass 9


### Jetson Xavier Tensorrt

In [ ]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + RTX 3090 Tensorrt in Docker

In [13]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")
print()
print(f"image_load_time_all {image_load_time_all}; {image_load_time_all/j} per one photo")
print(f"detect_bbox_time_all {detect_bbox_time_all}; {detect_bbox_time_all/j} per one photo")
print(f"craft_time_all {craft_time_all}; {craft_time_all/j} per one photo")
print(f"perspective_align_time_all {perspective_align_time_all}; {perspective_align_time_all/j} per one photo")
print(f"classification_time_all {classification_time_all}; {classification_time_all/j} per one photo")
print(f"ocr_time_all {ocr_time_all}; {ocr_time_all/j} per one photo")

Processed 310 photos
Time 19.727702140808105
One photo process 0.06363774884131647 seconds

image_load_time_all 5.8344831466674805; 0.018820913376346712 per one photo
detect_bbox_time_all 2.6671009063720703; 0.008603551310877646 per one photo
craft_time_all 9.693742752075195; 0.03127013790991998 per one photo
perspective_align_time_all 0.9096670150756836; 0.0029344097260505923 per one photo
classification_time_all 0.21755051612854004; 0.0007017758584791614 per one photo
ocr_time_all 0.3819010257720947; 0.0012319387928132089 per one photo
